工商银行：ICBC ， 
中国银行：BOC ， 
农业银行：ABCHINA ， 
交通银行：BANKCOMM ， 
建设银行：CCB ， 
招商银行：CMBCHINA ， 
光大银行：CEBBANK ， 
浦发银行：SPDB ， 
兴业银行：CIB ， 
中信银行：ECITIC

In [1]:
import requests
import json
import time
import pandas as pd

In [2]:
# supress InsecureRequestWarning
requests.packages.urllib3.disable_warnings()

In [3]:
class Forex:
    def __init__(self):
        self.APP_CODE = "6108e3cbd8f14444a1a6821d9e335c65"
        self.cache_alive = 60 * 60 * 1 # 1 hour
        self.required_currencies = ['USD', 'HKD', 'JPY', 'GBP', 'EUR', 'AUD', 'CAD', 'SGD']
    
    def cache_rates(self, bank_code, rates):
        with open(f"forex_cache/{bank_code}.json", "w") as f:
            json.dump({
                "timestamp": time.time(),
                "rates": rates
            }, f)

    def get_cached_rates(self, bank_code):
        try:
            with open(f"forex_cache/{bank_code}.json", "r") as f:
                data = json.load(f)
                if time.time() - data["timestamp"] < self.cache_alive:
                    return data["rates"]
        except:
            pass
        return None


    def ask_ali(self, bank_code):
        url = "https://ali-waihui.showapi.com/bank10"
        params = {
            "bankCode": bank_code
        }
        headers = {
            "Authorization": f"APPCODE {self.APP_CODE}"
        }

        response = requests.get(url, params=params, headers=headers, verify=False)

        if response.status_code == 200:
            result = response.json()["showapi_res_body"]["codeList"]

            result = self._format_rates(result)
            self.cache_rates(bank_code, result)

            return result
        else:
            raise Exception(f"Failed to get forex from {bank_code}")

    def get_cib_hyrs_rates(self):
        # ask ali for normal cib rates
        res = self.ask_ali("CIB")
        df = pd.DataFrame(res)

        # calculate huan yu ren sheng discounted rates by
        # sell = (buy+sell)/2 + ((sell-buy)/2)*0.5 = (buy+sell)/2 + (sell-buy)/4 = (buy+3*sell)/4
        # buy = (buy+sell)/2 - ((sell-buy)/2)*0.5 = (buy+sell)/2 - (sell-buy)/4 = (3*buy+sell)/4
        # cols exchangeRateCurrency	transferBuyingRate	transferSellingRate	notesBuyingRate	notesSellingRate
        discounted_df = pd.DataFrame({
            'exchangeRateCurrency': df['exchangeRateCurrency'],
            'transferBuyingRate': (3*pd.to_numeric(df['transferBuyingRate'], errors="coerce") + pd.to_numeric(df['transferSellingRate'], errors="coerce"))/4,
            'transferSellingRate': (pd.to_numeric(df['transferBuyingRate'], errors="coerce") + 3*pd.to_numeric(df['transferSellingRate'], errors="coerce"))/4,
            'notesBuyingRate': (3*pd.to_numeric(df['notesBuyingRate'], errors="coerce") + pd.to_numeric(df['notesSellingRate'], errors="coerce"))/4,
            'notesSellingRate': (pd.to_numeric(df['notesBuyingRate'], errors="coerce") + 3*pd.to_numeric(df['notesSellingRate'], errors="coerce"))/4,
        })

        discounted_df = discounted_df.sort_values(by='exchangeRateCurrency')
        discounted_df = discounted_df.reset_index(drop=True)
        res = discounted_df.to_dict(orient="records")
        self.cache_rates("CIB_HYRS", res)
        return res
        
    
    def get_hsbc_rates(self):
        res = requests.get("https://www.services.cn-banking.hsbc.com.cn/mobile/channel/digital-proxy/cnyTransfer/ratesInfo/remittanceRate",
             headers={'Content-Type': 'application/json'}, params={'locale': 'en_CN'}).json()["data"]["counterForRepeatingBlock"]
        df = pd.DataFrame(res)
        
        df_filtered = df[df['exchangeRateCurrency'].isin(self.required_currencies)]
        formatted_df = pd.DataFrame({
            'exchangeRateCurrency': df_filtered['exchangeRateCurrency'],
            'transferBuyingRate': (1/pd.to_numeric(df_filtered['transferBuyingRate'], errors="coerce")).round(7),
            'transferSellingRate': (1/pd.to_numeric(df_filtered['transferSellingRate'], errors="coerce")).round(7),
            'notesBuyingRate': (1/pd.to_numeric(df_filtered['notesBuyingRate'], errors="coerce")).round(7),
            'notesSellingRate': (1/pd.to_numeric(df_filtered['notesSellingRate'], errors="coerce")).round(7),
        })

        formatted_df = formatted_df.sort_values(by='exchangeRateCurrency')
        formatted_df = formatted_df.reset_index(drop=True)
        res = formatted_df.to_dict(orient="records")
        self.cache_rates("HSBC", res)
        return res

        
    def get_rates(self, bank_code, use_cache=True):
        assert bank_code in ["ICBC", "BOC", "ABCHINA", "BANKCOMM", "CCB", 
                             "CMBCHINA", "CEBBANK", "SPDB", "CIB", "CIB_HYRS", "ECITIC", "HSBC"]
        if use_cache:
            rates = self.get_cached_rates(bank_code)
            if rates:
                return rates
        
        if bank_code == "HSBC":
            return self.get_hsbc_rates()
        
        if bank_code == "CIB_HYRS":
            return self.get_cib_hyrs_rates()


        return self.ask_ali(bank_code)

    
    def _format_rates(self, rates):
        df = pd.DataFrame(rates)

    
        # Filter the DataFrame to include only the required currencies
        df_filtered = df[df['code'].isin(self.required_currencies)]
        
        # Create a new DataFrame with the desired columns
        formatted_df = pd.DataFrame({
            'exchangeRateCurrency': df_filtered['code'],
            'transferBuyingRate': pd.to_numeric(df_filtered['hui_in'], errors="coerce")/100,
            'transferSellingRate': pd.to_numeric(df_filtered['hui_out'], errors="coerce")/100,
            'notesBuyingRate': pd.to_numeric(df_filtered['chao_in'], errors="coerce")/100,
            'notesSellingRate': pd.to_numeric(df_filtered['chao_out'], errors="coerce")/100,
        })

        # sort the DataFrame by the required currencies
        formatted_df = formatted_df.sort_values(by='exchangeRateCurrency')
        
        formatted_df = formatted_df.reset_index(drop=True)
        return formatted_df.to_dict(orient="records")
    

In [6]:
forex = Forex()
res = forex.get_rates("HSBC", use_cache=False)
pd.DataFrame(res)

KeyboardInterrupt: 

In [5]:
forex = Forex()
res = forex.get_rates("CIB_HYRS", use_cache=False)
pd.DataFrame(res)

,exchangeRateCurrency,transferBuyingRate,transferSellingRate,notesBuyingRate,notesSellingRate
0,AUD,4.739200,4.758200,4.628800,4.721400
1,CAD,5.334600,5.356000,5.210325,5.314575
2,EUR,7.819850,7.851150,7.637675,7.790425
3,GBP,9.125700,9.162300,8.913150,9.091450
4,HKD,0.922575,0.924325,0.916975,0.922525
5,JPY,0.047600,0.047800,0.046475,0.047425
6,SGD,5.350900,5.372300,5.226175,5.330725
7,USD,7.222150,7.235850,7.178400,7.221800
